In [1]:
%%capture
!pip install monai
!pip install pynrrd
!pip intall torchsummary

In [2]:
from monai.data import Dataset
from monai.data import DataLoader, decollate_batch
from monai.transforms import (
    Activations,
    Activationsd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    Invertd,
    LoadImaged,
    LoadImage,
    MapTransform,
    NormalizeIntensityd,
    NormalizeIntensity,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    Spacingd,
    EnsureTyped,
    EnsureChannelFirstd,
    Resized,
    Resize,
    RandGaussianNoised,
    RandBiasFieldd,
    RandAdjustContrastd,
    RandGaussianSmoothd,
    RandRicianNoised,
    CropForegroundd,
    CropForeground,
    RandRotated,
    Rotate90d,
    Flipd,
    ScaleIntensityd,
    CenterSpatialCropd,
    KeepLargestConnectedComponent,
    FillHoles
)
import torch
from torch import nn
import torch.nn.functional as F
import os
import shutil
import tempfile
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import torch
from tqdm import tqdm
from torchvision.models import convnext_tiny, swin_t
from torchvision.models import efficientnet_v2_s
from sklearn.metrics import roc_auc_score
from sklearn.metrics  import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import brier_score_loss
import nrrd
from matplotlib.colors import ListedColormap
green = ListedColormap(['k', 'g'])
red = ListedColormap(['k', 'r'])
from monai.losses import DiceFocalLoss
from monai.metrics import DiceMetric
import pickle as pkl

2024-04-03 10:14:43.901040: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 10:14:43.901157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 10:14:44.095350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
images = os.listdir('/kaggle/input/scl-chest-ct/images')
masks = os.listdir('/kaggle/input/scl-chest-ct/masks')
dicts = []
for i in range (1, 18):
  dicts.append({
      'image': f'/kaggle/input/scl-chest-ct/images/{i}.nii',
      'label': f'/kaggle/input/scl-chest-ct/masks/{i}.nrrd'
  })

In [4]:
class LoadNRRD(MapTransform): # load NRRD files (labels) and one-hot encode
  def __call__(self, data):
    for key in self.keys:
      data[key] = torch.tensor(nrrd.read(data[key])[0])
      # data[key] = torch.stack([torch.eq(data[key], i)  for i in range(1, 5)], dim=0).to(torch.float32) # for multiclass
      # data[key] = torch.where(data[key] != 0, torch.tensor(1.0), torch.tensor(0.0)) # for lung segmentation
#       data[key] = torch.stack([
#           torch.where(data[key] == 1, torch.tensor(1.0), torch.tensor(0.0)),
#           torch.where(data[key] > 1, torch.tensor(1.0), torch.tensor(0.0))
#       ]).to(torch.float32)
    return data

In [5]:
preprocessing = Compose([
        LoadImaged(keys=["image"]),
        LoadNRRD(keys=['label']),
        # AddChannelDim(keys=['image', 'label']), # for lung segmentation
#         AddChannelDim(keys=['image']), # for multiclass
#         CropForegroundd(keys=['image', 'label'], source_key='image'),
#         Resized(keys=['image', 'label'], spatial_size=(128, 128, 32), mode=['trilinear', 'nearest-exact']), #expected efficientNetv2 size
        # Rotate90d(k=3, keys=['image', 'label']),
        # Rotate90d(k=1, keys=['image', 'label'], spatial_axes = (1, 2)),
        # Rotate90d(k=3, keys=['image', 'label']),
        # Rotate90d(k=2, keys=['image', 'label'], spatial_axes = (0, 2)),
        # Flipd(keys=['image'], spatial_axis=1), #because jpeg files don't have correct orientation metadata
        # ScaleIntensityd(keys='image', dtype=None), #for some reason slows preprocessing down
#         NormalizeIntensityd(keys="image"), #using mean and sd values from PyTorch doc
        # RandScaleIntensityd(keys="image", factors=0.1, prob=0.5),
        # RandGaussianNoised(keys='image', prob=0.5, std=0.05),
        # RandBiasFieldd(keys='image', degree=2),
        # RandAdjustContrastd(keys='image', prob=0.3, gamma=(0.95, 1.05)),
        # RandGaussianSmoothd(keys='image', prob=0.05, sigma_x=(0.95, 1.05), sigma_y=(0.95, 1.05)),
    ])

# Label shapes
17: Nl, EsFib, FFib


In [6]:
sub = dicts[-1]
nr = nrrd.read(sub['label'])

In [7]:
def get_names(sub):
    nr = nrrd.read(sub['label'])
    segment_names = {}
    for key, value in nr[1].items():
        if 'Segment' in key and '_Name' in key:
            segment_names[key] = value
    names = segment_names.values()
    names = [x for x in names if x != '0']
    return names

In [8]:
prep = preprocessing(dicts)

In [9]:
for subject in dicts:
    names = get_names(subject)
    sp = preprocessing(subject)
    uniques = torch.unique(sp['label'])
    print (subject)
    print (names)
    print (uniques)
    print (sp['label'].shape)
    print ('*' * 10)

{'image': '/kaggle/input/scl-chest-ct/images/1.nii', 'label': '/kaggle/input/scl-chest-ct/masks/1.nrrd'}
['Normal', 'GGO', 'Fibrosis', 'Fine fibrosis']
tensor([0, 1, 3, 4], dtype=torch.uint8)
torch.Size([512, 512, 198])
**********
{'image': '/kaggle/input/scl-chest-ct/images/2.nii', 'label': '/kaggle/input/scl-chest-ct/masks/2.nrrd'}
['Normal', 'GGO', 'Fibrosis', 'Fine Fibrosis']
tensor([0, 1, 2, 3], dtype=torch.int16)
torch.Size([512, 512, 120])
**********
{'image': '/kaggle/input/scl-chest-ct/images/3.nii', 'label': '/kaggle/input/scl-chest-ct/masks/3.nrrd'}
['Normal', 'GGO', 'Fibrosis', 'Fine fibrosis']
tensor([0, 1, 3], dtype=torch.uint8)
torch.Size([512, 512, 52])
**********
{'image': '/kaggle/input/scl-chest-ct/images/4.nii', 'label': '/kaggle/input/scl-chest-ct/masks/4.nrrd'}
['Normal', 'GGO', 'Fibrosis', 'Fine fibrosis']
tensor([0, 1, 2, 4], dtype=torch.uint8)
torch.Size([512, 512, 70])
**********
{'image': '/kaggle/input/scl-chest-ct/images/5.nii', 'label': '/kaggle/input/scl-

In [10]:
class AddChannelDim(MapTransform): #Add a channel dimension for images and one-hot encode
  def __call__(self, data):
    for key in self.keys:
      data[key] = data[key].unsqueeze(0)
      # if key == 'image':
        # data[key] = torch.cat([data[key],  data[key],  data[key]], dim=0) #since our pretrained model expects inputs with 3 channels
    return data

In [11]:
loaded = []

## Subjects 1-8:

In [12]:
class LoadNRRD(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
    def __call__(self, data):
        for key in self.keys:
            loaded_tensor = torch.tensor(nrrd.read(data[key])[0])
            # Add channel dimension (C, H, W, D)
            loaded_tensor = loaded_tensor.unsqueeze(0)
            # One-hot encode
            one_hot_tensor = torch.zeros((3,) + loaded_tensor.shape[1:])
            one_hot_tensor[0] = torch.where(loaded_tensor == 1, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[1] = torch.where(loaded_tensor == 3, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[2] = torch.where((loaded_tensor == 2) | (loaded_tensor == 4), torch.tensor(1.0), torch.tensor(0.0))
            data[key] = one_hot_tensor.to(torch.float32)
#             print(data[key].shape)
        return data

In [13]:
preprocessing = Compose([
        LoadImaged(keys=["image"]),
        AddChannelDim(keys=['image']),
        LoadNRRD(keys=['label']),
#         CropForegroundd(keys=['image', 'label'], source_key='image'),
        Resized(keys=['image', 'label'], spatial_size=(128, 128, 64), mode=['trilinear', 'nearest-exact']),
    ])

In [14]:
indexes = [1,2,3,4,5,7,8]
subjects = []
for index in indexes:
  subjects.append({
      'image': f'/kaggle/input/scl-chest-ct/images/{index}.nii',
      'label': f'/kaggle/input/scl-chest-ct/masks/{index}.nrrd'
  })
loaded.extend(preprocessing(subjects))

# Subjects 11, 14, 17
* 11, 14, 17: 1, 3, 4
* 9, 10, 13, 16: 1, 4
* 12, 15: 1, 2, 3

In [15]:
class LoadNRRD(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
    def __call__(self, data):
        for key in self.keys:
            loaded_tensor = torch.tensor(nrrd.read(data[key])[0])
            # Add channel dimension (C, H, W, D)
            loaded_tensor = loaded_tensor.unsqueeze(0)
            # One-hot encode
            one_hot_tensor = torch.zeros((3,) + loaded_tensor.shape[1:])
            one_hot_tensor[0] = torch.where(loaded_tensor == 1, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[1] = torch.where(loaded_tensor == 3, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[2] = torch.where(loaded_tensor == 4, torch.tensor(1.0), torch.tensor(0.0))
            data[key] = one_hot_tensor.to(torch.float32)
        return data

In [16]:
preprocessing = Compose([
        LoadImaged(keys=["image"]),
        AddChannelDim(keys=['image']),
        LoadNRRD(keys=['label']),
#         CropForegroundd(keys=['image', 'label'], source_key='image'),
        Resized(keys=['image', 'label'], spatial_size=(128, 128, 64), mode=['trilinear', 'nearest-exact']),
    ])
indexes = [11, 14, 17]
subjects = []
for index in indexes:
  subjects.append({
      'image': f'/kaggle/input/scl-chest-ct/images/{index}.nii',
      'label': f'/kaggle/input/scl-chest-ct/masks/{index}.nrrd'
  })
loaded.extend(preprocessing(subjects))

# Subjects 9, 10, 13
* 11, 14, 17: 1, 3, 4
* 9, 10, 13, 16: 1, 4
* 12, 15: 1, 2, 3

In [17]:
class LoadNRRD(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
    def __call__(self, data):
        for key in self.keys:
            loaded_tensor = torch.tensor(nrrd.read(data[key])[0])
            # Add channel dimension (C, H, W, D)
            loaded_tensor = loaded_tensor.unsqueeze(0)
            # One-hot encode
            one_hot_tensor = torch.zeros((3,) + loaded_tensor.shape[1:])
            one_hot_tensor[0] = torch.where(loaded_tensor == 1, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[1] = torch.where(loaded_tensor == 3, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[2] = torch.where(loaded_tensor == 4, torch.tensor(1.0), torch.tensor(0.0))
            data[key] = one_hot_tensor.to(torch.float32)
        return data

In [18]:
preprocessing = Compose([
        LoadImaged(keys=["image"]),
        AddChannelDim(keys=['image']),
        LoadNRRD(keys=['label']),
#         CropForegroundd(keys=['image', 'label'], source_key='image'),
        Resized(keys=['image', 'label'], spatial_size=(128, 128, 64), mode=['trilinear', 'nearest-exact']), #expected efficientNetv2 size

    ])
indexes = [9, 10, 13, 16]
subjects = []
for index in indexes:
  subjects.append({
      'image': f'/kaggle/input/scl-chest-ct/images/{index}.nii',
      'label': f'/kaggle/input/scl-chest-ct/masks/{index}.nrrd'
  })
loaded.extend(preprocessing(subjects))

# Subjects 12, 15
* 11, 14, 17: 1, 3, 4
* 9, 10, 13, 16: 1, 4
* 12, 15: 1, 2, 3

In [19]:
class LoadNRRD(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
    def __call__(self, data):
        for key in self.keys:
            loaded_tensor = torch.tensor(nrrd.read(data[key])[0])
            # Add channel dimension (C, H, W, D)
            loaded_tensor = loaded_tensor.unsqueeze(0)
            # One-hot encode
            one_hot_tensor = torch.zeros((3,) + loaded_tensor.shape[1:])
            one_hot_tensor[0] = torch.where(loaded_tensor == 1, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[1] = torch.where(loaded_tensor == 2, torch.tensor(1.0), torch.tensor(0.0))
            one_hot_tensor[2] = torch.where(loaded_tensor == 3, torch.tensor(1.0), torch.tensor(0.0))
            data[key] = one_hot_tensor.to(torch.float32)
        return data

In [20]:
preprocessing = Compose([
        LoadImaged(keys=["image"]),
        AddChannelDim(keys=['image']),
        LoadNRRD(keys=['label']),
#         CropForegroundd(keys=['image', 'label'], source_key='image'),
        Resized(keys=['image', 'label'], spatial_size=(128, 128, 64), mode=['trilinear', 'nearest-exact']), #expected efficientNetv2 size

    ])
indexes = [12, 15]
subjects = []
for index in indexes:
  subjects.append({
      'image': f'/kaggle/input/scl-chest-ct/images/{index}.nii',
      'label': f'/kaggle/input/scl-chest-ct/masks/{index}.nrrd'
  })
loaded.extend(preprocessing(subjects))

# Subject 6

In [21]:
class LoadNRRD(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
    def __call__(self, data):
        for key in self.keys:
            loaded_tensor = torch.tensor(nrrd.read(data[key])[0])
            # Add channel dimension (C, H, W, D)
            loaded_tensor = loaded_tensor
            # One-hot encode
            one_hot_tensor = torch.zeros((3,) + loaded_tensor.shape[1:])
            one_hot_tensor[0] = loaded_tensor[0, :, :, :]
            one_hot_tensor[1] = torch.zeros_like(loaded_tensor[0, :, :, :])
            one_hot_tensor[2] = loaded_tensor[1, :, :, :]
            data[key] = one_hot_tensor.to(torch.float32)
        return data

In [22]:
preprocessing = Compose([
        LoadImaged(keys=["image"]),
        AddChannelDim(keys=['image']),
        LoadNRRD(keys=['label']),
#         CropForegroundd(keys=['image', 'label'], source_key='image'),
        Resized(keys=['image', 'label'], spatial_size=(128, 128, 64), mode=['trilinear', 'nearest-exact']), #expected efficientNetv2 size

    ])
indexes = [6]
subjects = []
for index in indexes:
  subjects.append({
      'image': f'/kaggle/input/scl-chest-ct/images/{index}.nii',
      'label': f'/kaggle/input/scl-chest-ct/masks/{index}.nrrd'
  })
loaded.extend(preprocessing(subjects))

In [23]:
len(loaded)

17

In [24]:
with open('dataset.pkl', "wb") as file:
    pkl.dump(loaded, file)